In [ ]:
import sys
# Install gurobi
!{sys.executable} -m pip install -i https://pypi.gurobi.com gurobipy
!{sys.executable} -m pip install plotly geopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB, quicksum

# These are ours
import problem
import visualization

In [ ]:
def normalize(i, j):
    assert i != j
    return (i, j) if i < j else (j, i)

## Problem configuration

In [ ]:
from problem import C, D, cities, B

# What scenario we will run
scenario = 0

C = problem.C
D = problem.D
cities = problem.cities
B = problem.B
demand = B[scenario]

print(f'|C| = {len(cities)}, D = {len(D)} x {len(D[0])}, B = {len(B)} x {len(B[0])}, scenario = {scenario}')

In [ ]:
C = 10
D = D[:C, :C]
cities = cities[:C]
demand = demand[:C]

In [ ]:
# 1 if the core net should be a cycle, 0 if it should be a path
Z = 1
# The number of cities in the core net
NC = 4

## Model formulation

In [ ]:
# The set of cities (indices)
CITIES = list(range(C))
# The set of undirected edges
EDGES = [(i, j) for i, j in product(CITIES, repeat=2) if j > i]

In [ ]:
# The optimization model
model = gp.Model('autonomax')

In [ ]:
is_control_center = model.addVars(CITIES, vtype=GRB.BINARY, name='is-control-center')

# There is exactly one control center
model.addConstr(quicksum(is_control_center[c] for c in CITIES) == 1);

In [ ]:
# Whether or not the edge between city `i` and `j` is part of the core net
is_core_edge = model.addVars(EDGES, vtype=GRB.BINARY, name='is-core-edge')

# Whether or not a city `i` is part of the core net
is_core_city = model.addVars(C, vtype=GRB.BINARY, name='is-core-city')

# The control center is required to be one of the core cities
model.addConstrs(
    (is_core_city[i] >= is_control_center[i] for i in CITIES), 
    name='control-center-directly-connected'
)

# Force cc to 0 if a city has no adjacent core edges
model.addConstrs(
    (is_core_city[i] <= sum(is_core_edge[normalize(i, j)] for j in CITIES if j != i) for i in CITIES),
    name='core-city-ub'
)

# Force cc to 1 if a city has an adjacent core edge
model.addConstrs(
    (is_core_city[i] >= is_core_edge[normalize(i, j)] for i, j in product(CITIES, repeat=2) if i != j),
    name='core-city-lb'
)

# A cycle has |V| = |E|. A path has |V| = |E| + 1
model.addConstr(sum(is_core_city[i] for i in CITIES) == sum(is_core_edge[i, j] for i, j in EDGES) + 1 - Z)

# Ensure that all core nodes have at most degree 2
model.addConstrs(
    (2 * is_core_city[i] >= sum(is_core_edge[normalize(i, j)] for j in CITIES if j != i) for i in CITIES),
    name='disallow-core-tree',
)

# Ensure the number of core cities equals NC
model.addConstr(
    sum(is_core_city[i] for i in CITIES) == NC, 
    name='exactly-nc-core-cities'
);

In [ ]:
T = NC

is_connected_step = model.addVars(C, T, vtype=GRB.BINARY, name='is-connected-step')
is_connectable_step = model.addVars(C, C, range(1, T), vtype=GRB.BINARY, name='is-connectable-step')

# At timestep 0, the control center is the only connected city
model.addConstrs(
    (is_connected_step[i, 0] == is_control_center[i] for i in CITIES),
    name='control-center-is-connected',
)

# A city `i` is connectable to a city `j` at time `t` if (i, j) ∈ core edges U sub edges and `j`
# is connected at time `t - 1`
model.addConstrs(
    (2 * is_connectable_step[i, j, t] <= is_connected_step[i, t - 1] + is_connected_step[j, t - 1] + is_core_edge[i, j] for t, (i, j) in product(range(1, T), EDGES)), 
    name = 'is-connectable'
)

# At timestep n, a city can be marked as connected if it has a core edge to a city 
# that was set as connected on timestep (n - 1)
model.addConstrs(
    (is_connected_step[i, t] <= sum(is_connectable_step[(*normalize(i, j), t)] for j in CITIES if j != i) for t, i in product(range(1, T), CITIES)),
    name='is-connected-timestep'
)

# Ensure that all core cities are connected at some timestep
model.addConstrs(
    (sum(is_connected_step[i, t] for t in range(T)) == is_core_city[i] for i in CITIES),
    name='connected-graph'
);

In [ ]:
# How much (directed) demand is sent along an edge between two cities
flow = model.addVars(CITIES, CITIES, name='flow')
# What direction a flow is sent in. For i < j: 1 = forward (i -> j), 0 = backward (j -> i)
direction = model.addVars(EDGES, vtype=GRB.BINARY, name='flow-direction')

# We can only run flow in a single direction
model.addConstrs(
    (flow[i, j] <= sum(demand) * direction[i, j] for (i, j) in EDGES),
    name='flow-dir-forward'
)

# We can only run flow in a single direction
model.addConstrs(
    (flow[j, i] <= sum(demand) * (1 - direction[i, j]) for (i, j) in EDGES),
    name='flow-dir-backward'
)

# The required surplus must equal the difference between (ingoing + self demand) - (outgoing)
model.addConstrs(
    (sum(demand) * is_control_center[i] == demand[i] + sum(flow[j, i] for j in CITIES if j != i) - sum(flow[i, j] for j in CITIES if j != i) for i in CITIES),
    name='conservation-of-flow'
);

In [ ]:
# Whether an edge is a sub edge, i.e. part of the sub network.
is_sub_edge = model.addVars(EDGES, vtype=GRB.BINARY, name='is-sub-edge')

# We can only send flow along an edge if it is either a sub edge or a core edge
model.addConstrs(
    (sum(demand) * (is_sub_edge[e] + is_core_edge[e]) >= flow[e] + flow[tuple(reversed(e))] for e in EDGES),
    name='force-is-edge-up-if-flow',
    
)

# Force is_sub_edge to 0 if there is no flow (with M = min(demand))
model.addConstrs(
   (min(demand) * is_sub_edge[e] <= flow[e] + flow[tuple(reversed(e))] for e in EDGES),
    name='force-is-edge-down-if-no-flow',
)

# An edge can not be both a core edge and a sub edge
model.addConstrs(
    (is_core_edge[e] + is_sub_edge[e] <= 1 for e in EDGES),
    name='disallow-is-both-sub-and-core',
)

# The cost of an edge excluding its cost as a core edge
edge_cost = model.addVars(EDGES, name='edge-cost')

# The edge cost must be bounded below by 10 + (0.1 D_ij)**1.5 * B if it is a sub edge
M = lambda e: 10 + (0.1 * D[e])**1.5 * sum(demand)
model.addConstrs(
    (edge_cost[e] + M(e) * is_core_edge[e] >= 10 * is_sub_edge[e] + (0.1 * D[e])**1.5 * (flow[e] + flow[tuple(reversed(e))]) for e in EDGES),
    name='edge-cost-lb'
);

In [ ]:
model.setObjective(sum(edge_cost[e] + 10 * D[e] * is_core_edge[e] for e in EDGES), GRB.MINIMIZE)
model.Params.timeLimit = 600.0
model.Params.MIPFocus = 2
model.optimize()

In [ ]:
relaxation = model.relax()
relaxation.optimize()

In [ ]:
for v in model.getVars():
    if abs(v.x) < 1e-8 or 'is-connect' in v.varName or 'flow-direction' in v.varName or 'is-sub' in v.varName:
        continue
    
    print(f'{v.varName} = {v.x}')

In [ ]:
sum(v.x for v in edge_cost.values())

In [ ]:
def non_zero(items):
    for (key, v) in items.items():
        if abs(v.x) > 1e-9:
            yield key
            
def city_output():
    core_city = list(non_zero(is_core_city))
    control_center = list(non_zero(is_control_center))
    
    return [{
            'Index': i,
            'Name': cities[i],
            'IsControlCenter': i in control_center,
            'Demand': demand[i],
            'IngoingFlow': sum(flow[j, i] for j in CITIES if j != i).getValue(),
            'OutgoingFlow': sum(flow[i, j] for j in CITIES if j != i).getValue(),
        } for i in CITIES]
    
            
def edge_output():
    core = list(non_zero(is_core_edge))
    utilized = list(non_zero(flow))
    
    return [{
        'From': cities[i],
        'To': cities[j],
        'Type': 'CORE' if normalize(i, j) in core else 'SUB',
        'Flow': abs(flow[i, j].x - flow[j, i].x),
        'Cost': (10 * D[i, j] * is_core_edge[normalize(i, j)] + edge_cost[normalize(i, j)]).getValue(),
        'Distance': D[i, j],
    } for (i, j) in set(core + utilized)]

In [ ]:
pd.DataFrame(city_output())

In [ ]:
pd.DataFrame(edge_output())

In [ ]:
visualization.show(city_output(), edge_output())